In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from sklearn.preprocessing import StandardScaler
import pandas as pd

# ------------------------
# Load and Preprocess Data
# ------------------------

print("Loading data...")
train_df = pd.read_csv('/content/customer_churn_dataset-training-master.csv')
test_df = pd.read_csv('/content/customer_churn_dataset-testing-master.csv')
train_df = train_df.dropna()

column_mapping = {
    'CustomerID': 'CustomerID',
    'Age': 'Age',
    'Gender': 'Gender',
    'Tenure': 'Tenure',
    'Usage Frequency': 'Usage_Frequency',
    'Support Calls': 'Support_Calls',
    'Payment Delay': 'Payment_Delays',
    'Subscription Type': 'Subscription',
    'Contract Length': 'Contract_Length',
    'Total Spend': 'Total_Spend',
    'Last Interaction': 'Last_Interaction',
    'Churn': 'Churn'
}
train_df = train_df.rename(columns=column_mapping)
test_df = test_df.rename(columns={
    'Usage Frequency': 'Usage_Frequency',
    'Support Calls': 'Support_Calls',
    'Payment Delay': 'Payment_Delays',
    'Subscription Type': 'Subscription',
    'Contract Length': 'Contract_Length',
    'Total Spend': 'Total_Spend',
    'Last Interaction': 'Last_Interaction'
})

# Encode categorical columns
categorical_cols = ['Gender', 'Subscription', 'Contract_Length']
train_encoded = pd.get_dummies(train_df, columns=categorical_cols, drop_first=True)
test_encoded = pd.get_dummies(test_df, columns=categorical_cols, drop_first=True)

# Match columns
train_feature_columns = [col for col in train_encoded.columns if col != 'Churn']
for col in train_feature_columns:
    if col not in test_encoded.columns:
        test_encoded[col] = 0
test_columns_to_use = [col for col in test_encoded.columns if col in train_feature_columns or col == 'Churn']
test_encoded = test_encoded[test_columns_to_use]

# Drop ID columns
train_encoded = train_encoded.drop('CustomerID', axis=1)
test_encoded = test_encoded.drop('CustomerID', axis=1)

# Extract features/targets
X_train = train_encoded.drop('Churn', axis=1)
y_train = train_encoded['Churn'].values

if 'Churn' in test_encoded.columns:
    X_test = test_encoded.drop('Churn', axis=1)
    y_test = test_encoded['Churn'].values
else:
    X_test = test_encoded
    y_test = None

# Standardize features
print("Standardizing...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ------------------------
# Define the ANN Model
# ------------------------

model = Sequential()

# Input layer and first hidden layer
model.add(Dense(10, kernel_initializer='normal', activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dropout(0.1))
model.add(BatchNormalization())

# Second hidden layer
model.add(Dense(7, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization())

# Output layer
model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))  # single unit for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# ------------------------
# Train the Model
# ------------------------

model_history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=10,
    batch_size=32,
    verbose=1
)


Loading data...
Standardizing...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 48s 4ms/step - accuracy: 0.9327 - loss: 0.1799 - val_accuracy: 0.9957 - val_loss: 0.0401
Epoch 2/10
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 76s 3ms/step - accuracy: 0.9680 - loss: 0.0967 - val_accuracy: 1.0000 - val_loss: 0.0222
Epoch 3/10
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 37s 3ms/step - accuracy: 0.9694 - loss: 0.0933 - val_accuracy: 0.9995 - val_loss: 0.0610
Epoch 4/10
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 46s 4ms/step - accuracy: 0.9695 - loss: 0.0927 - val_accuracy: 0.9993 - val_loss: 0.0328
Epoch 5/10
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 35s 3ms/step - accuracy: 0.9707 - loss: 0.0900 - val_accuracy: 1.0000 - val_loss: 0.0263
Epoch 6/10
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9712 - loss: 0.0897 - val_accuracy: 0.9997 - val_loss: 0.0260
Epoch 7/10
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 41s 3ms/step - accuracy: 0.9719 - loss: 0.0889 - val_accuracy: 0.9999 - val_loss: 0.0272
Epoch 8/10
11021/11021 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 

In [ ]:
!pip install streamlit
# ngrok.set_auth_token("2w2ozqDgpv74NEtqkPz4CXoYkpI_2S9QMVhBdNcvHLmGbgAdB")
!pip install streamlit tensorflow scikit-learn pandas numpy matplotlib



In [ ]:
%%writefile app.py
# [ Paste the entire Streamlit code here ]

import streamlit as st
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Sample ANN model creation
def build_ann(input_dim):
    model = Sequential()
    model.add(Dense(16, input_dim=input_dim, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
    return model

# Dummy training (simulate a pretrained model)
@st.cache_resource
def train_model():
    # Simulate dummy training data
    np.random.seed(42)
    X_train = np.random.rand(500, 14)
    y_train = np.random.randint(0, 2, 500)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    model = build_ann(X_train.shape[1])
    model.fit(X_train_scaled, y_train, epochs=10, verbose=0)

    return model, scaler

# User input preprocessing
def preprocess_input(user_input):
    input_df = pd.DataFrame([user_input])

    input_df['Gender_Male'] = 1 if user_input['Gender'] == 'Male' else 0
    input_df['Subscription_Basic'] = 1 if user_input['Subscription'] == 'Basic' else 0
    input_df['Subscription_Premium'] = 1 if user_input['Subscription'] == 'Premium' else 0
    input_df['Subscription_Standard'] = 1 if user_input['Subscription'] == 'Standard' else 0
    input_df['Contract_Length_Annual'] = 1 if user_input['Contract_Length'] == 'Annual' else 0
    input_df['Contract_Length_Monthly'] = 1 if user_input['Contract_Length'] == 'Monthly' else 0
    input_df['Contract_Length_Quarterly'] = 1 if user_input['Contract_Length'] == 'Quarterly' else 0

    input_df.drop(columns=['Gender', 'Subscription', 'Contract_Length'], inplace=True)

    feature_order = [
        'Age', 'Tenure', 'Usage_Frequency', 'Support_Calls', 'Payment_Delays',
        'Total_Spend', 'Last_Interaction', 'Gender_Male',
        'Subscription_Basic', 'Subscription_Premium', 'Subscription_Standard',
        'Contract_Length_Annual', 'Contract_Length_Monthly', 'Contract_Length_Quarterly'
    ]
    return input_df[feature_order]

# Streamlit UI
def main():
    st.set_page_config(page_title="ANN Churn Predictor", layout="centered")
    st.title("🔮 Customer Churn Prediction using ANN")
    st.write("Fill out the customer details below to predict churn probability.")

    model, scaler = train_model()

    with st.form("churn_form"):
        col1, col2, col3 = st.columns(3)
        age = col1.number_input("Age", 18, 100, 30)
        gender = col2.selectbox("Gender", ["Male", "Female"])
        tenure = col3.number_input("Tenure (months)", 0, 120, 12)

        usage_frequency = col1.number_input("Usage Frequency", 0, 100, 10)
        support_calls = col2.number_input("Support Calls", 0, 50, 5)
        payment_delays = col3.number_input("Payment Delays", 0, 100, 10)

        total_spend = col1.number_input("Total Spend", 0.0, 10000.0, 500.0)
        last_interaction = col2.number_input("Last Interaction (days)", 0, 365, 30)

        subscription = col3.selectbox("Subscription", ["Basic", "Standard", "Premium"])
        contract = col1.selectbox("Contract Length", ["Monthly", "Quarterly", "Annual"])

        submitted = st.form_submit_button("Predict")

        if submitted:
            input_data = {
                'Age': age,
                'Gender': gender,
                'Tenure': tenure,
                'Usage_Frequency': usage_frequency,
                'Support_Calls': support_calls,
                'Payment_Delays': payment_delays,
                'Total_Spend': total_spend,
                'Last_Interaction': last_interaction,
                'Subscription': subscription,
                'Contract_Length': contract
            }

            processed_input = preprocess_input(input_data)
            scaled_input = scaler.transform(processed_input)
            prediction = model.predict(scaled_input)[0][0]

            st.subheader("📊 Prediction Result")
            st.write(f"**Churn Probability:** `{prediction:.2f}`")
            if prediction > 0.5:
                st.error("⚠️ The customer is likely to churn.")
            else:
                st.success("✅ The customer is likely to stay.")

if __name__ == "__main__":
    main()


Overwriting app.py


In [ ]:
!pip install streamlit


In [ ]:
!pip install pyngrok streamlit


In [ ]:
from pyngrok import ngrok

# Open a tunnel on the default Streamlit port 8501
public_url = ngrok.connect(port=8501)
print(f"Streamlit app URL: {public_url}")

# Write app
with open("app.py", "w") as f:
    f.write("""
# Paste your app.py code here
""")

# Launch
!streamlit run app.py &


ERROR:pyngrok.process.ngrok:t=2025-04-22T14:14:15+0000 lvl=eror msg="failed to reconnect session" obj=tunnels.session err="authentication failed: The authtoken you specified is properly formed, but it is invalid.\nYour authtoken: 2w2ozqDgpv74NEtqkPz4CXoYkpI_2S9QMVhBdNcvHLmGbgAdB\nThis usually happens when:\n    - You reset your authtoken\n    - Your authtoken was for a team account that you were removed from\n    - You are using ngrok link and this credential was explicitly revoked\nGo to your ngrok dashboard and double check that your authtoken is correct:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_107\r\n"
ERROR:pyngrok.process.ngrok:t=2025-04-22T14:14:15+0000 lvl=eror msg="session closing" obj=tunnels.session err="authentication failed: The authtoken you specified is properly formed, but it is invalid.\nYour authtoken: 2w2ozqDgpv74NEtqkPz4CXoYkpI_2S9QMVhBdNcvHLmGbgAdB\nThis usually happens when:\n    - You reset your authtoken\n    - Your authtoken was 

PyngrokNgrokError: The ngrok process errored on start: authentication failed: The authtoken you specified is properly formed, but it is invalid.\nYour authtoken: 2w2ozqDgpv74NEtqkPz4CXoYkpI_2S9QMVhBdNcvHLmGbgAdB\nThis usually happens when:\n    - You reset your authtoken\n    - Your authtoken was for a team account that you were removed from\n    - You are using ngrok link and this credential was explicitly revoked\nGo to your ngrok dashboard and double check that your authtoken is correct:\nhttps://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_107\r\n.